In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel #parallel computation
import torch.optim as optim #optimizer 
import torch.utils.data
from torch.autograd import Variable #stochastic gradient descent

In [2]:
!ls

 AItRBM-proof.pdf	    boltzmann_machine.py   ml-100k.zip
'boltzmann machine.ipynb'   ml-100k		   ml-1m


In [3]:
!ls ml-1m/

movies.dat   ratings.dat  test_set.csv	    Train_Test_Set_Creation.R
ratings.csv  README	  training_set.csv  users.dat


In [4]:
!head ml-1m/movies.dat

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
6::Heat (1995)::Action|Crime|Thriller
7::Sabrina (1995)::Comedy|Romance
8::Tom and Huck (1995)::Adventure|Children's
9::Sudden Death (1995)::Action
10::GoldenEye (1995)::Action|Adventure|Thriller


In [5]:
#importing the dataset
movies = pd.read_csv("ml-1m/movies.dat", sep="::", header = None, engine="python",encoding="latin-1")
#some movie titles have special characters and utf8 dont read them. Thats the encoding option


In [6]:
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [7]:
users = pd.read_csv("ml-1m/users.dat", sep="::", header = None, engine="python",encoding="latin-1")
users
#user id, gender, age, code to user job, zip code

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [8]:
ratings = pd.read_csv("ml-1m/ratings.dat", sep="::", header = None, engine="python",encoding="latin-1")
ratings
#user, movie, rating (1 to 5), and a extra column that we dont care

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [9]:
### preparing training and test set

In [10]:
training_set = pd.read_csv("ml-100k/u1.base", delimiter="\t")
training_set
#user, movie, rating, timestamps (no relevant)

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [11]:
#turning df in array
training_set = np.array(training_set, dtype="int")
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [12]:
#test set
test_set = pd.read_csv("ml-100k/u1.test", delimiter="\t")
test_set = np.array(test_set, dtype="int")

In [13]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [14]:
# getting the number of users and movies, because then we will do 
# matrices of users and movies, were value will be the rate

nb_users = int(max(max(training_set[:,0]), max(test_set[:,0]))) # nb number
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))


In [15]:
# converting the data into an array with users in lines and movies 
# in columns. no ratings are equal to 0

def convert(data):
    #as we will use torch, we will do a 2d np array, we will do a
    #list of list
    new_data  = []
    for id_users in range(1,nb_users+1): #upper bound is excludede, for this is +1
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings# -1 cause indexes start a 0
        new_data.append(list(ratings)) #list of list as torch expect
    return new_data



In [16]:
training_set = convert(training_set)
test_set = convert(test_set)
    

In [17]:
#converting list of list into torch tensors
#tensor are  arrays that contains elements of a single datatype
#is a multidimensional matrix, instead of np array, this is
#a pythorch array much more efficient that np array for nn
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

#done the preprocessing

In [18]:
#boltzmann machine output are 1 or 0 
#converting the ratings into binary ratings: 1 (liked) or 0 (not liked)

training_set[training_set == 0] = -1
#all zeroes in the original training set not exist (not rated), and
#as the rating are 0 or 1, the original zeroes need another
#value (not rating)
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [19]:
#making the architecture of the neural network

# a class with three functions, initialize the RBM, sample H that will
# sample the probabilities of the hidden nodes given the visible nodes
# and sample_v that will sample the probabilities of the visible nodes
# given the hidden nodes
class RBM():
    def __init__(self, nv, nh):
        #self is the object that will be created (for this and not 
        #global variables)
        #nv: number of visible nodes
        #nh: number of hidden nodes
        
        #initializing all the parameters to optimize during the 
        #RBM training process (weights and bias)
        self.W = torch.randn(nh, nv) # weight, a torch tensor
        #these weights are the parameters of the probabilities of the
        #visible nodes given the hidden nodes
        #rand initialize weights of probb P of the visible nodes 
        #given the hidden nodes (with size nh, nv)
        #according normal distribution (mean 0 and variance 1)
        
        self.a = torch.randn(1, nh)
        #bias for probb of hidden nodes given visible P(h,v)
        self.b = torch.randn(1, nv)
        #bias for probb of visible nodes given hidden P(v,h)
        
    #sampling hidden nodes according P(h,v) that is equal to the
    #sigmoid activation function
    
    #first function of gibbs sampling
    def sample_h(self, x): #x correspond to the visible neurons, v in P(h,v) 
        #computing the P(h,v), thats is the probb that hidden neuron
        #equals 1 (liked) given value of the visible neuron that is actually
        #our input vector of observations
        
        #P(H,v) = sigmoid activation function applied to wx + a (bias of
        #hidden nodes, b is for visible)
        wx = torch.mm(x, self.W.t()) #W^(t)*x
        activation = wx + self.a.expand_as(wx) #wx + a = activation (what is inside activation function)
        #expand_as(wx) is to do the minibatches
        p_h_given_v = torch.sigmoid(activation) 
        
        #we are doing a bernoulli RBM because we are predicting a binary
        #outcome, and so what we will return also is some bernoulli
        #samples of that distribution. This meand that right now P(h|v)
        #is a vector of nh elements (probb of the Ith element is activated)
        #given visible nodes, so now we want to sample the activation of
        #each of this nodes  (yes or not), and taking as example
        #a 70% probb of activate neuron (0.7), we take a random number
        #between 0 and 1, and if this random number is below 0.7, then
        #we will activate the neuron, otherwise we will not activate
        #the neuron, that how bernoulli sampling works

        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    
    #similar to the previous, but now for P(v|h)
    def sample_v(self, y): #y are hidden nodes
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    #v0: vector of rating movies by one user
    #vk: visible nodes obtained after k samplings
    #ph0: vector of probb that at the 1rst iteration the hidden
    #nodes are equal to one given the values of V0
    #Phk: probb of hidden nodes after K samplings given the values
    #of the visible nodes Vk
    def train(self, v0, vk, ph0, phk):
        #aproximating RBM loglikehood gradient
        #for the previous, the cotrastive divergence algorithm
        #is a good option to train (comes with gibbs sampling)
        
        #optimizing weights to minimize energy as described the algorithm
        #in the paper
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)  
    

In [20]:
nv = len(training_set[0])
nh = 100 #number of features that the RBM will be detect
batch_size = 100
rbm = RBM(nv, nh)

In [21]:
#training the RBM
nb_epoch = 10
#we will update the weights after the observations of each batch 
#passed through the network, and in the end we will get our visible 
#nodes with the new ratings for the movies that were not originally
#rated
for epoch in range(1, nb_epoch + 1): 
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size]
        v0 = training_set[id_user : id_user + batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3222)
epoch: 2 loss: tensor(0.2364)
epoch: 3 loss: tensor(0.2460)
epoch: 4 loss: tensor(0.2490)
epoch: 5 loss: tensor(0.2467)
epoch: 6 loss: tensor(0.2463)
epoch: 7 loss: tensor(0.2482)
epoch: 8 loss: tensor(0.2453)
epoch: 9 loss: tensor(0.2472)
epoch: 10 loss: tensor(0.2463)


In [23]:
"""## Testing the RBM"""

test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1] #we do not replace, cause are visible nodes obtained after k samplings
    vt = test_set[id_user:id_user+1] # original rating in the test set
    #we actually need to keep the training set, cause is the input 
    #that will be used to activate hidden neurons to get the output.
    #right now the training set contains the ratings of training set
    #and it does not contain the answers of the test set, but by using the inputs
    #of the training set we will activate neurons of the RBM to predict
    #the ratings of the movies that were not rated yet
    #and that is the ratings of the test set. So we need this as input to
    #get the predicted ratings of the test set
    
    
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2406)
